In [1]:
# from ANEMO import ANEMO
# from ANEMO import read_edf
import numpy as np

import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import bayesianchangepoint as bcp

In [2]:
%run ../../0_parameters.ipynb


WELCOME

2020-03-25T17:10:29+01:00

CPython 3.7.7
IPython 7.13.0

numpy 1.18.2
matplotlib 3.2.1
torch 1.4.0

compiler   : Clang 11.0.0 (clang-1100.0.33.17)
system     : Darwin
release    : 19.0.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit
host name  : ekla
Git hash   : 753891cd04d4b65fab288ecc8e590d3dd84ab4f3
Git repo   : https://github.com/chloepasturel/AnticipatorySPEM
Git branch : master


In [3]:
print('fig_width =', fig_width)

fig_width = 30


In [4]:
%mkdir -p results_enregistrement

In [5]:
%ls ../../data

bet_s01_2017-09-29_123528.pkl     eyeMvt_s05_2017-10-03_143803.EDF
bet_s02_2017-09-25_142503.pkl     eyeMvt_s05_2017-10-03_143803.asc
bet_s03_2017-09-28_154353.pkl     eyeMvt_s05_2017-10-03_143803.pkl
bet_s04_2017-09-26_162348.pkl     eyeMvt_s06_2017-09-28_115250.asc
bet_s05_2017-09-26_143606.pkl     eyeMvt_s06_2017-09-28_115250.edf
bet_s06_2017-09-26_122431.pkl     eyeMvt_s06_2017-09-28_115250.pkl
bet_s07_2017-09-19_152205.pkl     eyeMvt_s07_2017-09-20_151043.asc
bet_s08_2017-10-24_105728.pkl     eyeMvt_s07_2017-09-20_151043.edf
bet_s09_2017-11-07_105527.pkl     eyeMvt_s07_2017-09-20_151043.pkl
bet_s10_2017-11-14_102614.pkl     eyeMvt_s08_2017-10-26_121823.asc
bet_s11_2017-11-08_155132.pkl     eyeMvt_s08_2017-10-26_121823.edf
bet_s12_2017-11-06_164857.pkl     eyeMvt_s08_2017-10-26_121823.pkl
eyeMvt_s01_2017-10-23_100057.asc  eyeMvt_s09_2017-11-08_094717.asc
eyeMvt_s01_2017-10-23_100057.edf  eyeMvt_s09_2017-11-08_094717.edf
eyeMvt_s01_2017-10-23_100057.pkl  eyeMvt_s09_2017-11-08_094717

In [6]:
subjects = ['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12']

import glob
liste = {'bet':{}, 'eyeMvt':{}}
for fname in glob.glob('../../data/*pkl'):
    print(fname.split('/'))
    a = fname.split('/')[-1].split('.')[0].split('_')
    liste[a[0]][a[1]] = a[2]+'_'+a[3]

BET, EYEMVT = {}, {}
for s in liste['bet'].keys() :
    if s in subjects :
        a = '../../data/bet_%s_%s.pkl'%(s, liste['bet'][s])
        with open(a, 'rb') as fichier : BET[s] = pickle.load(fichier, encoding='latin1')
        a = '../../parametre/fct_velocity_sigmo/param_Fit_%s_fct_velocity_sigmo.pkl'%s
        with open(a, 'rb') as fichier : EYEMVT[s] = pickle.load(fichier, encoding='latin1')

# ---------------------------------------------------
param_exp = BET['s08']

['..', '..', 'data', 'bet_s07_2017-09-19_152205.pkl']
['..', '..', 'data', 'bet_s03_2017-09-28_154353.pkl']
['..', '..', 'data', 'eyeMvt_s04_2017-09-27_161040.pkl']
['..', '..', 'data', 'eyeMvt_s01_2017-10-23_100057.pkl']
['..', '..', 'data', 'bet_s09_2017-11-07_105527.pkl']
['..', '..', 'data', 'eyeMvt_s02_2017-09-26_095637.pkl']
['..', '..', 'data', 'eyeMvt_s09_2017-11-08_094717.pkl']
['..', '..', 'data', 'bet_s01_2017-09-29_123528.pkl']
['..', '..', 'data', 'bet_s02_2017-09-25_142503.pkl']
['..', '..', 'data', 'eyeMvt_s05_2017-10-03_143803.pkl']
['..', '..', 'data', 'eyeMvt_s06_2017-09-28_115250.pkl']
['..', '..', 'data', 'eyeMvt_s03_2017-10-03_134421.pkl']
['..', '..', 'data', 'bet_s08_2017-10-24_105728.pkl']
['..', '..', 'data', 'eyeMvt_s12_2017-11-17_172706.pkl']
['..', '..', 'data', 'bet_s05_2017-09-26_143606.pkl']
['..', '..', 'data', 'eyeMvt_s07_2017-09-20_151043.pkl']
['..', '..', 'data', 'bet_s12_2017-11-06_164857.pkl']
['..', '..', 'data', 'bet_s11_2017-11-08_155132.pkl']
[

In [7]:
import matplotlib.gridspec as gridspec

In [8]:
def plot_experiment(mode_bcp=None, tau=40, p=None,
                    fig=None, axs=None, fig_width=10, titre=None, t_titre=35, t_label=25,
                    color=[['k', 'k'], ['r', 'r'], ['k','w']], color_bet='r', color_va='k',
                    color_bcp=['darkgreen'], name_bcp=['$P_{BBCP}$'],
                    alpha = [[.35,.15],[.35,.15],[1,0]], lw=2.5):

    
    def plot_result_bcp(ax1, mode, observation, time, n_trial, name_bcp, color, name=True) :

        from scipy.stats import beta
        p_bar, r_bar, beliefs = bcp.inference(observation, h=h, p0=p0, r0=r0)
        p_hat, r_hat = bcp.readout(p_bar, r_bar, beliefs, mode=mode, fixed_window_size=40, p0=p0)
        p_low, p_sup = np.zeros_like(p_hat), np.zeros_like(p_hat)

        for i_trial in range(n_trial):
            p_low[i_trial], p_sup[i_trial] = beta.ppf([.05, .95], a=p_hat[i_trial]*r_hat[i_trial], b=(1-p_hat[i_trial])*r_hat[i_trial])
        ax1.plot(time, p_hat, c=color, lw=lw, alpha=1, label=name_bcp if name is True else '')
        ax1.plot(time, p_sup, c=color, lw=lw, alpha=1, ls='--')
        ax1.plot(time, p_low, c=color, lw=lw, alpha=1, ls='--')
        ax1.fill_between(time, p_sup, p_low, lw=.5, alpha=.5, facecolor=color)

        return ax1


    N_trials, N_blocks = param_exp['N_trials'], param_exp['N_blocks']
    h = 1./tau
    
    c = 'w'
    p = param_exp['p']
    ec = 0.1

    mini = 5
    ec1 = ec*mini*2
    
    
    def figure():
        #------------------------------------------------
        # FIGURE
        #------------------------------------------------
        fig, axs = plt.subplots(3+1, 1, figsize=(fig_width, ((3+0.5)*fig_width/3)/(1.6180)))

        gs1 = gridspec.GridSpec(1, 1)
        gs1.update(left=0+0.072, bottom=0.84, right=1-0.04, top=1.-0.11, hspace=0.05)
        axs[0] = plt.subplot(gs1[0])

        for card in ['bottom', 'top', 'right']: axs[0].spines[card].set_visible(False)
        axs[0].spines['left'].set_bounds(0, 1)

        gs2 = gridspec.GridSpec(3, 1)
        gs2.update(left=0+0.072, bottom=0+0.1, right=1-0.04, top=0.85-0.03, hspace=0.05)
        for a in range(3): axs[a+1] = plt.subplot(gs2[a])

        #------------------------------------------------
        # Y
        #------------------------------------------------
        for a in range(len(axs)):
            axs[a].set_ylim(-(ec/2), 1+ec-(ec/2))
            axs[a].set_yticks([0])
            if a>0: axs[a].yaxis.set_label_coords(-0.05, 0.5)

        axs[0].set_ylabel('TD', fontsize=t_label/1.3)
        axs[0].set_yticks([0, 1])
        axs[0].set_yticklabels(['left','right'],fontsize=t_label/3)    

        axs[1].set_ylabel('Probability', fontsize=t_label/1.5)
        axs[3].set_ylabel('Bet score', fontsize=t_label/1.5)

        for a in [1, 3] :
            axs[a].set_yticks([0, 0.5, 1])
            axs[a].set_yticklabels(['0', '0.5', '1'],fontsize=t_label/1.8)
            axs[a].yaxis.set_label_coords(-0.03, 0.5)

        axs[2].set_yticks([])
        ax2 = axs[2].twinx()
        ax2.set_ylabel('Anticipatory\neye velocity (°/s)', fontsize=t_label/1.5, color=c)
        ax2.set_ylim(-mini-(ec1/2), mini + ec1-(ec1/2))
        ax2.set_yticks([-mini, 0, mini])
        ax2.set_yticklabels(['-%s'%mini, '0', '%s'%mini],fontsize=t_label/1.8)
        ax2.yaxis.set_label_coords(-0.063, 0.5)
        ax2.yaxis.set_ticks_position('left')

        for card in ['left', 'top', 'bottom', 'right']: ax2.spines[card].set_color(c)
        ax2.tick_params(axis='both', colors=c, labelcolor=c)

        #------------------------------------------------
        # X
        #------------------------------------------------
        for a in range(len(axs)):
            #------------------------------------------------
            # cosmétique
            #------------------------------------------------
            axs[a].set_xlim(-1, N_trials)
            if a==(len(axs)-1) :
                axs[a].set_xticks([0, 50, 100, 150, 200])
                axs[a].set_xticklabels([0, 50, 100, 150, 200], ha='left', fontsize=t_label/1.8)
                axs[a].xaxis.set_ticks_position('bottom')
            else :
                axs[a].set_xticks([])
        axs[-1].set_xlabel('Trial #', fontsize=t_label)
        #----------------------------------------------------------------

        for a in axs :
            a.tick_params(axis='both', colors=c, labelcolor=c)
            a.xaxis.label.set_color(c)
            a.yaxis.label.set_color(c)
            for card in ['left', 'top', 'bottom', 'right']: a.spines[card].set_color(c)
        
        return fig, axs, ax2

    
    ###################################################################################################################################
    # PLOT
    ###################################################################################################################################
    p = BET[subjects[0]]['p']
    p0, r0 =  0.5, 1.0
    
    resusu = np.zeros((len(subjects), N_trials))
    v_antiti = np.zeros((len(subjects), N_trials))

    for x, y in enumerate(subjects) :
        resusu[x] = (BET[y]['results'][:, 0]+1)/2
        a_anti, start_anti, latency = EYEMVT[y]['a_anti'][0], EYEMVT[y]['start_anti'][0], EYEMVT[y]['latency'][0]
        v_antiti[x] = (np.array(a_anti)*((np.array(latency)-np.array(start_anti))/1000))

    results = np.median(resusu, axis=0)
    va = np.median(v_antiti, axis=0)

    results_low, results_sup = np.percentile(resusu, [25, 75], axis=0)
    va_low, va_sup = np.percentile(v_antiti, [25, 75], axis=0)

    
    for t in range(1, N_trials) :
        print('t =', t)
        fig, axs, ax2 = figure()

        for a in range(len(axs)):
            #------------------------------------------------
            # Barre Pause
            #------------------------------------------------
            for num_pause in range(1,4) :
                axs[a].bar(num_pause*50-1, 1+ec, bottom=-ec/2, color=c, width=.5, linewidth=0)

        
        
        ###################################################################################################################################
        axs[0].plot(np.arange(1, t), p[1:t, 0, 0], 'w.', ms=4)

        #-------------------------------------------------------------------------------------------------------------
        axs[0].step(range(t), p[:t, 0, 0], lw=1, c=color[0][0], alpha=alpha[0][0], where='mid')
        axs[0].fill_between(range(t), np.zeros_like(p[:t, 0, 0]), p[:t, 0, 0],
                            lw=.5, alpha=alpha[0][0], facecolor=color[0][0], step='mid')
        axs[0].fill_between(range(t), np.ones_like(p[:t, 0, 0]), p[:t, 0, 0],
                            lw=.5, alpha=alpha[0][1], facecolor=color[0][1], step='mid')

        for a in range(1, 4) :
            axs[a].step(range(N_trials), p[:, 0, 1], lw=lw, c=color[1][0], alpha=1, label='$P_{true}$')
            axs[a].plot(range(N_trials), 0.5*np.ones(N_trials), lw=1.5, c=c, alpha=0.5)

        #------------------------------------------------------------
        # BCP
        #------------------------------------------------------------

        for b, mode in enumerate(mode_bcp):
            if t < 51 :    liste = [0,t]
            elif t < 101 : liste = [0,50, t]
            elif t < 151 : liste = [0,50,100, t]
            else :         liste = [0,50,100,150, t]
            
            for pause in range(len(liste)-1) :
                if pause==0 : name=True
                else :        name=False
                n_trial = liste[pause+1]-liste[pause]
                axs[1] = plot_result_bcp(axs[1], mode, p[liste[pause]:liste[pause+1], 0, 0],
                                         np.arange(liste[pause], liste[pause+1]), n_trial, name_bcp[b], color_bcp[b],
                                         name=name)
        
        if t < 51 :    liste = [0,t]
        elif t < 101 : liste = [0,50, t]
        elif t < 151 : liste = [0,50,100, t]
        else :         liste = [0,50,100,150, t]
            
        for pause in range(len(liste)-1) :

            #------------------------------------------------------------
            # eyeMvt
            #------------------------------------------------------------
            va__ = va[liste[pause]:liste[pause+1]]
            va_sup__ = va_sup[liste[pause]:liste[pause+1]]
            va_low__ = va_low[liste[pause]:liste[pause+1]]
            titi = np.arange(liste[pause], liste[pause+1])

            ax2.plot(titi, va__, color=color_va, lw=lw, alpha=1)

            ax2.fill_between(titi, va_sup__, va_low__, lw=.5, alpha=.6, facecolor=color_va)
            ax2.plot(titi, va_sup__, c=color_va, lw=lw, alpha=.9, ls='--')
            ax2.plot(titi, va_low__, c=color_va, lw=lw, alpha=.9, ls='--')

            #------------------------------------------------------------
            # bet
            #------------------------------------------------------------
            results__ = results[liste[pause]:liste[pause+1]]
            results_sup__ = results_sup[liste[pause]:liste[pause+1]]
            results_low__ = results_low[liste[pause]:liste[pause+1]]

            axs[3].plot(titi, results__, color=color_bet, lw=lw, alpha=1)

            axs[3].fill_between(titi, results_sup__, results_low__, lw=.5, alpha=.6, facecolor=color_bet)
            axs[3].plot(titi, results_sup__, c=color_bet, lw=lw, alpha=.9, ls='--')
            axs[3].plot(titi, results_low__, c=color_bet, lw=lw, alpha=.9, ls='--')


        #------------------------------------------------
        leg = axs[1].legend(fontsize=t_label/1.8, bbox_to_anchor=(0., 1.33, 1, 0.), loc=3, ncol=4,
                            mode="expand", borderaxespad=0., framealpha=0)

        for line, text in zip(leg.get_lines(), leg.get_texts()): text.set_color(line.get_color())

        #fig.tight_layout()
        plt.subplots_adjust(hspace=0.05)
        fig.savefig('results_enregistrement/2_results_enregistrement_%s.png'%t, dpi=45,
                    transparent=True, bbox_inches='tight', pad_inches=0)
        
        plt.close()
        
        #plt.show()
        #------------------------------------------------

    #return fig, axs

In [9]:
N_trials = 200

color=[['w', 'w'], [color_true_p, color_true_p], ['w','none']]
alpha = [[.9,.5],[.0,.0],[1,0]]

#fig_width = fig_width/6
#t_label = t_label/6


opt = dict(color=color,alpha=alpha, titre='', t_label=t_label, lw=2,
           color_bet='dimgrey', color_va='w', fig_width=fig_width)
plot_experiment(mode_bcp=['leaky', 'mean'], color_bcp=[color_bcp_fix, color_bcp_mean],
                name_bcp=['$P_{leaky}$', '$P_{BBCP}$'], **opt)


t = 1
t = 2
t = 3
t = 4
t = 5
t = 6
t = 7
t = 8
t = 9
t = 10
t = 11
t = 12
t = 13
t = 14
t = 15
t = 16
t = 17
t = 18
t = 19
t = 20
t = 21
t = 22
t = 23
t = 24
t = 25
t = 26
t = 27
t = 28
t = 29
t = 30
t = 31
t = 32
t = 33
t = 34
t = 35
t = 36
t = 37
t = 38
t = 39
t = 40
t = 41
t = 42
t = 43
t = 44
t = 45
t = 46
t = 47
t = 48
t = 49
t = 50
t = 51
t = 52
t = 53
t = 54
t = 55
t = 56
t = 57
t = 58
t = 59
t = 60
t = 61
t = 62
t = 63
t = 64
t = 65
t = 66
t = 67
t = 68
t = 69
t = 70
t = 71
t = 72
t = 73
t = 74
t = 75
t = 76
t = 77
t = 78
t = 79
t = 80
t = 81
t = 82
t = 83
t = 84
t = 85
t = 86
t = 87
t = 88
t = 89
t = 90
t = 91
t = 92
t = 93
t = 94
t = 95
t = 96
t = 97
t = 98
t = 99
t = 100
t = 101
t = 102
t = 103
t = 104
t = 105
t = 106
t = 107
t = 108
t = 109
t = 110
t = 111
t = 112
t = 113
t = 114
t = 115
t = 116
t = 117
t = 118
t = 119
t = 120
t = 121
t = 122
t = 123
t = 124
t = 125
t = 126
t = 127
t = 128
t = 129
t = 130
t = 131
t = 132
t = 133
t = 134
t = 135
t = 136
t = 137
t = 138
t = 

In [12]:
%run 2020-03-18_Bet_result_enregistrement_generate_video.py

FPS =  59.92542652920312 framerate= 60


t:   0%|          | 0/407 [00:00<?, ?it/s, now=None]

Moviepy - Building video 2_results_enregistrement.mp4.
Moviepy - Writing video 2_results_enregistrement.mp4



Moviepy - Done !
Moviepy - video ready 2_results_enregistrement.mp4
236.5409 	WARNING 	Monitor specification not found. Creating a temporary one...
